In [1]:
import pandas as pd
from pytz import country_timezones
import time
import numpy as np
import timezonefinder
from datetime import datetime as dt
import pytz
import calendar
import seaborn as sns
%matplotlib inline
from matplotlib import rcParams

In [2]:
start_ini = time.time() #check time

In [3]:
import warnings
warnings.filterwarnings('ignore') #remove warning

In [4]:
#import dataset
df = pd.read_csv('https://media.githubusercontent.com/media/DBertazioli/NeoMeetup/master/csv/exported_queries/Distribuzione_temporale_eventi.csv')
df.head()

,name,time,country,state,lon,lat
0,Lunch meetup at Shanghai Slims 13th April Sat...,1.555128e+12,cn,NONE,121.47,31.23
1,"\t Meetup Salud M\u00f3vil, IA Salud - Observa...",1.554413e+12,co,NONE,-75.59,6.25
2,\t Rob Thomas: Chip Tooth Tour,1.560213e+12,us,CO,-105.08,39.7
3,\tHayden Island,1.554568e+12,us,OR,-122.77,45.44
4,\tNorth - University of Portland,1.555173e+12,us,OR,-122.77,45.44


In [5]:
df['state'] = df['state'].str.lower() #lower charcter 'NONE'-->'none'
df[df['state'] == 'none'].head()

,name,time,country,state,lon,lat
0,Lunch meetup at Shanghai Slims 13th April Sat...,1.555128e+12,cn,none,121.47,31.23
1,"\t Meetup Salud M\u00f3vil, IA Salud - Observa...",1.554413e+12,co,none,-75.59,6.25
5,\tPhunsukh Wangdu Isolated Camping,1.554532e+12,in,none,77.56,12.97
6,"\tSevern Shires Way part 4, Farnborough to Mid...",1.554628e+12,gb,none,-1.26,51.76
9,\tWine and Paint Party - Unicorn Painting,1.555206e+12,nz,none,174.76,-36.85


In [6]:
df.loc[df['state'] == 'none', 'state'] = df['country'] #fill the state's value with country's value if there was 'none'

In [7]:
df.isnull().sum() #check sum of nan fro columns

name        76
time       144
country    144
state      144
lon        144
lat        144
dtype: int64

In [8]:
df = df.dropna() #rm nan value for all df (rows)

In [9]:
df.isnull().any() #check

name       False
time       False
country    False
state      False
lon        False
lat        False
dtype: bool

In [10]:
#df.loc[df['state'] == 'none', 'state'] = df.fillna(value=pd.np.nan, inplace=True) 
#it's only if would take nan into country that do not have a state

In [11]:
#df.isnull().sum() #check

In [12]:
#df.isnull().any() #another check :/

In [13]:
#same method for check nan value into coordinates
#df['lon'] = df['lon'].str.lower() *****crea problemi con NAN su 180k rows di lat******
#df['lat'] = df['lat'].str.lower()
df.loc[df['lon'] == 'NONE', 'lon'] = df.fillna(value=pd.np.nan, inplace=True)
df.loc[df['lat'] == 'NONE', 'lat'] = df.fillna(value=pd.np.nan, inplace=True)
df.isnull().sum()

name        0
time        0
country     0
state       0
lon        98
lat         5
dtype: int64

In [14]:
df = df.dropna() #remove this nan
df.isnull().sum()

name       0
time       0
country    0
state      0
lon        0
lat        0
dtype: int64

In [15]:
df.tail() #df clean

,name,time,country,state,lon,lat
313432,~Vegan Drinks & Pizza~ at Bonnie BraeTavern,1.555726e+12,us,co,-104.99,39.75
313433,~~ Avengers - ENDGAME ~~,1.556664e+12,us,pa,-75.14,40
313434,~~ Dawn Mine Hike ~~,1.555168e+12,us,ca,-118.31,34.06
313435,~~ Lets explore Giant Ledge!,1.555247e+12,us,ny,-74.06,41.52
313436,~~ Pinochle PLUS......Game Night ~~~,1.555196e+12,us,pa,-75.14,40


In [16]:
df['lon'] = df['lon'].astype(float) #convert coordinates into float for problem into cell below (timezonefinder)
df['lat'] = df['lat'].astype(float)
df.dtypes

name        object
time       float64
country     object
state       object
lon        float64
lat        float64
dtype: object

In [17]:
df = df[[c for c in df if c not in ['lat','lon']] + ['lat','lon']] #change order of column for best position of lat and lng

In [18]:
df = df.reset_index(drop=True) #reset index for resolve eroor of out-bounded

In [19]:
tf = timezonefinder.TimezoneFinder() #defining time zones on lat, long data
df['timezone_str'] = "nan"
count = 0
start = time.time()
for line in df.itertuples():
    count+=1
    try:
        df.timezone_str.at[line.Index] = tf.timezone_at(lat = df.iloc[line.Index]['lat'],
                                                          lng = df.iloc[line.Index]['lon'])
    except Exception as e:
        print(e)
        break
    #if count == 10:
        #break
end = time.time()
print("task completed in "+str((end-start)/60)+" minutes")

task completed in 1.910099486509959 minutes


In [20]:
df[555:565] #check

,name,time,country,state,lat,lon,timezone_str
555,Data Showdown 2019,1.559115e+12,au,au,-33.87,151.21,Australia/Sydney
556,Database DevOps - Keep Your Delivery Processe...,1.557356e+12,us,nc,35.89,-78.64,America/New_York
557,Day Hiking Course - Conditioning Hiking Seri...,1.554944e+12,us,wa,46.99,-122.87,America/Los_Angeles
558,DeFi NYC Presents Investing in DeFi,1.554763e+12,us,ny,40.72,-74.00,America/New_York
559,"Dealing with stress, anxiety and work-life ba...",1.554313e+12,gb,gb,51.52,-0.10,Europe/London
560,Debate Meetup,1.556235e+12,ca,ab,51.04,-114.06,America/Edmonton
561,Deep Ellum Instagram Photowalk,1.557680e+12,us,tx,32.78,-96.78,America/Chicago
562,Deep Ellum Instagram Photowalk,1.555276e+12,us,tx,32.78,-96.78,America/Chicago
563,Deep Ellum Instagram Photowalk,1.555261e+12,us,tx,32.78,-96.78,America/Chicago
564,Delaware River Labor Day Weekend Camping,1.567188e+12,us,nj,40.79,-74.49,America/New_York


In [21]:
df.dtypes

name             object
time            float64
country          object
state            object
lat             float64
lon             float64
timezone_str     object
dtype: object

In [22]:
#this step is dated (only for convert apoc in neo)
#df['time'] = pd.to_datetime(df['time']) #convert time (object) into datettime type (needed fro convert after into timestamp)

In [23]:
##this step is dated (only for convert apoc in neo)
'''df['timestamp'] = df.fillna(value=pd.np.nan, inplace=True) #fill with nan value
start = time.time()
count = 0
for line in df.itertuples():
    count+=1
    try:
        df['timestamp'].at[line.Index] = dt.timestamp(df.iloc[line.Index]['time'])
    except Exception as e:
        print(e)
        break
    #if count == 100:
        #break
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')'''

"df['timestamp'] = df.fillna(value=pd.np.nan, inplace=True) #fill with nan value\nstart = time.time()\ncount = 0\nfor line in df.itertuples():\n    count+=1\n    try:\n        df['timestamp'].at[line.Index] = dt.timestamp(df.iloc[line.Index]['time'])\n    except Exception as e:\n        print(e)\n        break\n    #if count == 100:\n        #break\nend = time.time()\nprint('task completed in '+str((end-start)/60)+' minutes')"

In [24]:
df.head(10) #check

,name,time,country,state,lat,lon,timezone_str
0,Lunch meetup at Shanghai Slims 13th April Sat...,1.555128e+12,cn,cn,31.23,121.47,Asia/Shanghai
1,"\t Meetup Salud M\u00f3vil, IA Salud - Observa...",1.554413e+12,co,co,6.25,-75.59,America/Bogota
2,\t Rob Thomas: Chip Tooth Tour,1.560213e+12,us,co,39.70,-105.08,America/Denver
3,\tHayden Island,1.554568e+12,us,or,45.44,-122.77,America/Los_Angeles
4,\tNorth - University of Portland,1.555173e+12,us,or,45.44,-122.77,America/Los_Angeles
5,\tPhunsukh Wangdu Isolated Camping,1.554532e+12,in,in,12.97,77.56,Asia/Kolkata
6,"\tSevern Shires Way part 4, Farnborough to Mid...",1.554628e+12,gb,gb,51.76,-1.26,Europe/London
7,\tSpeak EZ Toastemasters,1.555024e+12,us,tx,32.96,-96.75,America/Chicago
8,\tSpeak EZ Toastemasters,1.554419e+12,us,tx,32.96,-96.75,America/Chicago
9,\tWine and Paint Party - Unicorn Painting,1.555206e+12,nz,nz,-36.85,174.76,Pacific/Auckland


In [25]:
def rounder(t): #this function round the time to nearest hour
    if t.minute >= 30:
        if t.hour == 23:
            return t.replace(second=0, microsecond=0, minute=0, hour=0)
        else:
            return t.replace(second=0, microsecond=0, minute=0, hour= t.hour+1)
    else:
        return t.replace(second=0, microsecond=0, minute=0)

In [26]:
df['datetime_timezoned'] = df.fillna(value=pd.np.nan, inplace=True) #fill with nan value

In [27]:
df['datetime_timezoned'] = df['datetime_timezoned'].astype(object) #convert for resolve error in the cell above (should not be a number)
df.dtypes

name                   object
time                  float64
country                object
state                  object
lat                   float64
lon                   float64
timezone_str           object
datetime_timezoned     object
dtype: object

In [28]:
start = time.time()
count = 0
for line in df.itertuples():
    count+=1
    try:
        df.datetime_timezoned.at[line.Index] = rounder((dt.fromtimestamp
                                                   (timestamp = (df.iloc[line.Index]['time'])/1000,
                                                    tz = pytz.timezone(df.iloc[line.Index]['timezone_str'])
                                                   ))).strftime("%d/%m/%Y %H:%M:%S")
    except Exception as e:
        print(e)
        break
    #if count == 100:
        #break
end = time.time()
print('task completed in '+str((end-start)/60)+' minutes')

task completed in 1.7703694184621175 minutes


In [29]:
df = df[[c for c in df if c not in ['timezone_str','time','datetime_timezoned']] + ['timezone_str','time','datetime_timezoned']]
df.head(10) #re-order column

,name,country,state,lat,lon,timezone_str,time,datetime_timezoned
0,Lunch meetup at Shanghai Slims 13th April Sat...,cn,cn,31.23,121.47,Asia/Shanghai,1.555128e+12,13/04/2019 12:00:00
1,"\t Meetup Salud M\u00f3vil, IA Salud - Observa...",co,co,6.25,-75.59,America/Bogota,1.554413e+12,04/04/2019 17:00:00
2,\t Rob Thomas: Chip Tooth Tour,us,co,39.70,-105.08,America/Denver,1.560213e+12,10/06/2019 19:00:00
3,\tHayden Island,us,or,45.44,-122.77,America/Los_Angeles,1.554568e+12,06/04/2019 10:00:00
4,\tNorth - University of Portland,us,or,45.44,-122.77,America/Los_Angeles,1.555173e+12,13/04/2019 10:00:00
5,\tPhunsukh Wangdu Isolated Camping,in,in,12.97,77.56,Asia/Kolkata,1.554532e+12,06/04/2019 12:00:00
6,"\tSevern Shires Way part 4, Farnborough to Mid...",gb,gb,51.76,-1.26,Europe/London,1.554628e+12,07/04/2019 10:00:00
7,\tSpeak EZ Toastemasters,us,tx,32.96,-96.75,America/Chicago,1.555024e+12,11/04/2019 18:00:00
8,\tSpeak EZ Toastemasters,us,tx,32.96,-96.75,America/Chicago,1.554419e+12,04/04/2019 18:00:00
9,\tWine and Paint Party - Unicorn Painting,nz,nz,-36.85,174.76,Pacific/Auckland,1.555206e+12,14/04/2019 14:00:00


In [30]:
df.to_csv('I:/Massi/Documents/GitHub/NeoMeetup/csv/exported_queries/Distribuzione_temporale_eventi_fix.csv') #csv

In [31]:
end_fin = time.time()
print("script completed in "+str((end_fin-start_ini)/60)+" minutes")

script completed in 3.812490542729696 minutes
